# Load user data

These instructions show how to load user data into the Senzing engine.

In this exercise:

1. You upload a file of records to be inserted into Senzing.
1. Data source names are extracted from the file of records.
1. The data source names are added to the Senzing configuration.
1. Records from the file of records are added to Senzing.

**Warning**:
Remember that the notebooks are not permanent when using the Playground docker container. 
You can save a notebook to your workstation by selecting <b>File</b> > <b>Download</b> in Jupyter Lab.

## Upload file of records

The following instructions show how to upload a file of JSON lines.
Here is an [example file] to download.

1. In upper-left corner of JuypterLab, click on the
   "Upload Files" icon
   ()
1. Choose the file of JSON lines to upload.
   When successful the file is seen in the left-hand file list.
1. Modify the value of <code>fileName</code> to match your file name.

[example file]: https://raw.githubusercontent.com/senzing-garage/playground/refs/heads/main/rootfs/examples/notebooks/python/senzing-example-data.json

## Prepare Go enviroment

Define global imports, types, and variables.

In [ ]:
import (
	"google.golang.org/grpc"
	"google.golang.org/grpc/credentials/insecure"
    "github.com/senzing-garage/sz-sdk-go-grpc/szabstractfactory"
    "github.com/senzing-garage/sz-sdk-go/senzing"
)

type DataSourceKey struct {
    Data_Source string
}

type Record struct {
    Data_Source string
    Record_ID   string
}

var (
    ctx = context.TODO()
    fileName = "senzing-example-data.json"
    err error
	grpcAddress = "localhost:8261"
    jsonDataSource DataSourceKey
    jsonRecord Record
    homePath = "./"
)

## Define functions

Create a function for testing error conditions.

In [ ]:
func testErr(err error) {
    if err != nil {
        panic(err)
    }
}

Create a function for getting an SzAbstractFactory that talks over gRPC.

In [ ]:
func getSzAbstractFactory() senzing.SzAbstractFactory {
    grpcConnection, err := grpc.NewClient(grpcAddress, grpc.WithTransportCredentials(insecure.NewCredentials()))
    testErr(err)
    return &szabstractfactory.Szabstractfactory{
    	GrpcConnection: grpcConnection,
    }
}

Create a function for extracting "DATA_SOURCE" values from JSON lines in files.

In [ ]:
func extractDataSources(filePath string) []string {
	result := []string{}
	file, err := os.Open(filePath)
	if err != nil {
		panic(err)
	}
	defer file.Close()

	scanner := bufio.NewScanner(file)
	for scanner.Scan() {
		line := scanner.Bytes()
		err := json.Unmarshal(line, &jsonDataSource)
		testErr(err)
		if !slices.Contains(result, jsonDataSource.Data_Source) {
			result = append(result, jsonDataSource.Data_Source)
		}
	}

	if err := scanner.Err(); err != nil {
		log.Fatal(err)
	}
	return result
}

Create a function to add DataSource names to Senzing configuration.

In [ ]:
func addDatasourcesToSenzingConfig(szAbstractFactory senzing.SzAbstractFactory, dataSourceNames []string) error {

	szConfig, err := szAbstractFactory.CreateConfig(ctx)
	if err != nil {
		return err
	}

	szConfigManager, err := szAbstractFactory.CreateConfigManager(ctx)
	if err != nil {
		return err
	}

	oldConfigID, err := szConfigManager.GetDefaultConfigID(ctx)
	if err != nil {
		return err
	}

	oldJsonConfig, err := szConfigManager.GetConfig(ctx, oldConfigID)
	if err != nil {
		return err
	}

	configHandle, err := szConfig.ImportConfig(ctx, oldJsonConfig)
	if err != nil {
		return err
	}

	for _, value := range dataSourceNames {
		_, err := szConfig.AddDataSource(ctx, configHandle, value)
		if err != nil {
			fmt.Println(err)
		}
	}

	newJsonConfig, err := szConfig.ExportConfig(ctx, configHandle)
	if err != nil {
		return err
	}

	newConfigID, err := szConfigManager.AddConfig(ctx, newJsonConfig, "Add TruthSet datasources")
	if err != nil {
		return err
	}

	err = szConfigManager.ReplaceDefaultConfigID(ctx, oldConfigID, newConfigID)
	if err != nil {
		return err
	}

	err = szAbstractFactory.Reinitialize(ctx, newConfigID)
	if err != nil {
		return err
	}

	return nil
}

Create a function to add records to Senzing from a file of JSON lines.

In [ ]:
func addRecords(szAbstractFactory senzing.SzAbstractFactory, filepath string) error {
	file, err := os.Open(filepath)
	if err != nil {
		return err
	}
	defer file.Close()

	szEngine, err := szAbstractFactory.CreateEngine(ctx)
	if err != nil {
		return err
	}

	scanner := bufio.NewScanner(file)
	for scanner.Scan() {
		line := scanner.Bytes()
		err := json.Unmarshal(line, &jsonRecord)
		testErr(err)
		result, err := szEngine.AddRecord(ctx, jsonRecord.Data_Source, jsonRecord.Record_ID, string(line), senzing.SzWithInfo)
		testErr(err)
		fmt.Println(result)
	}
	return nil
}

Create a function to pretty print JSON.

In [ ]:
func asPrettyJSON(str string) string {
	var prettyJSON bytes.Buffer
	if err := json.Indent(&prettyJSON, []byte(str), "", "    "); err != nil {
		return str
	}
	return prettyJSON.String()
}

## Main

Add records by calling previously defined functions.

In [ ]:
%%
// User input.

inputFile := fmt.Sprintf("%s%s", homePath, fileName)

// Create Senzing gRPC client.

grpcConnection, err := grpc.NewClient(grpcAddress, grpc.WithTransportCredentials(insecure.NewCredentials()))
testErr(err)
szAbstractFactory := &szabstractfactory.Szabstractfactory{
	GrpcConnection: grpcConnection,
}

// Identify datasources and update Senzing configuration.

dataSourceNames := extractDataSources(inputFile)
fmt.Printf("Found the following DATA_SOURCE values in the data: %v\n", dataSourceNames)

err = addDatasourcesToSenzingConfig(szAbstractFactory, dataSourceNames)
testErr(err)

// Add records.

err = addRecords(szAbstractFactory, inputFile)
testErr(err)